In [1]:
import cv2
import numpy as np

In [2]:
#Parameters for Shi-Tomashi corner detection
st_params=dict(maxCorners=30, qualityLevel=0.2, minDistance=2, blockSize=7)

#Parameters for Lucas Kande Optical flow
lk_params=dict(winSize=(15,15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,1))


In [4]:
#Video Capture
cap= cv2.VideoCapture('Vechile.mp4')

#Color for optical flow
color= (0,255,0)

#Read the capture and get the first frame
ret, first_frame= cap.read()
prev_gray= cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

#Find the strongest corner in the first frame
prev= cv2.goodFeaturesToTrack(prev_gray, mask= None, **st_params)

#create the image with the same dimmension as the frame for the later drawing purposes
mask= np.zeros_like(first_frame)

#Loop over the frames
while(cap.isOpened()):
    ret, frame= cap.read()
    gray= cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #Calculate the optical flow by Lucas- Kanade 
    next, status, error= cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev, None, **lk_params)
    
    #select good features for previous position
    good_old= prev[status==1]
    
    #elect good features for next position
    good_new= next[status==1]
    
    #Draw optical flow track
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        #return coordinates for new and old position
        a,b= new.ravel()
        c,d= old.ravel()
        
        #Draw line between old and new position
        mask= cv2.line(mask, (a,b), (c,d), color, 2)
        
        #Draw a filled circle on the corners 
        frame= cv2.circle(frame, (a,b), 3, color, -1)
    
    #Overlay the optical flow on original image
    output= cv2.add(frame, mask)
    
    #Update previous frame
    prev_gray= gray.copy()
    
    #Update good feature
    prev= good_new.reshape(-1,1,2)
    
    #Display the frame
    cv2.imshow('Optical Flow Video', output)
    
    if cv2.waitKey(300)& 0xFF== ord('q'):
        break

cv2.destroyAllWindows()
cap.release()